In [ ]:
import numpy as np
import time
import json
import http.client, urllib.parse
import tweepy
import pickle
import tweepy
from os import listdir
import pickle
import re
from os.path import isfile, join, isdir

In [ ]:
mypath = "/content/drive/MyDrive/CS245/after10/"
outputpath = "/content/drive/MyDrive/CS245/hey/"

In [ ]:
onlyfiles = []
for f in listdir(mypath):
  if (isdir(join(mypath, f))):
    newpath = join(mypath, f)
    temp = [join(newpath, t) for t in listdir(newpath) if isfile(join(newpath, t))]
    onlyfiles.extend(temp)
  elif (isfile(join(mypath, f))):
    onlyfiles.append(join(mypath, f))

In [ ]:
def get_mean(x):
  x = np.array(x).reshape(4,2)
  mean_val = np.mean(x, axis = 0)
  return mean_val

In [ ]:
us_lat_max = 49.5904
us_lat_min = 24.9493
us_long_max = -66.9326
us_long_min = -125.0011

In [ ]:
loc = np.array([-100,30]).reshape(1,2)
not_in_us(loc)

0

In [ ]:
def not_in_us(loc):
  #loc 0,0 is longitude and 0,1 is latitude
  long = loc[0,0]
  lat = loc[0,1]
  if (long>us_long_min and long<us_long_max):
    if (lat>us_lat_min and lat<us_lat_max):
      return 0
  return 1

In [ ]:
def preprocess(text):
  text = text.replace('\r','').replace('\n',' ') #remove new lines
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) #remove urls
  text = re.sub(r'[^\w\d\s\@\#\.\,\:\;\-]*', '', text) #remove non alphanumeric
  return text

In [ ]:
def add_to_dataset(dataset, itemlist, dic, us_filt = False):
  for item in itemlist:
    user = item['user']
    uniq = user.id
    if (uniq in dic):
      continue
    else:
      dic[uniq] = 1
    id = user.screen_name
    #print(user.screen_name)
    bow = ""
    if (item['type']==1):
      location = item['location'].coordinates['coordinates']
      print(location)
    elif (item['type']==2):
      location = get_mean(item['location'].place.bounding_box.coordinates)
    else:
      x = str(user.location)
      if (x.lower() == "none"):
        print("oops")
        continue
      if (re.search(r'([^A-Za-z\s\,\.])+',x)) is not None:
        # removes locations that contain special characters
        print("oops")
        continue
      if (re.search(r'^(([A-Z]+([a-z\,\.])*)+(\s)*)+$',x)) is None:
        #removes locations not in title case
        print("oops")
        continue
      location = item['location']
    #print(location)
    location = np.array(location).reshape(1,2)
    if (us_filt and not_in_us(location)):
      print("not in us")
      continue
    loc = str(location[0,1])+'\t'+str(location[0,0])
    #print(loc)
    for tweet in item['tweets']:
      text = preprocess(tweet.text)
      #text= tweet.text.replace('\r','').replace('\n',' ')
      bow+=text+' ||| '
    line = id+'\t'+loc+'\t'+bow
    #line = bytes(line, "utf-8").decode("unicode_escape") # python3 
    dataset.append(line)
  return 0
  #print(line)

In [ ]:
dataset = []
dic = {}
print(len(onlyfiles))
for f in onlyfiles:
  print("######", f, "######")
  with open (f, 'rb') as fp:
    itemlist = pickle.load(fp)
    a = add_to_dataset(dataset, itemlist, dic, us_filt=True)

In [ ]:
with open(outputpath+"dataset_final_us_1.total", "w") as fp:
  for line in dataset:
    fp.write(str(line)+"\n")

In [ ]:
len(dataset)

8222

In [ ]:
with open(outputpath+"dataset_update.total", 'r') as fp:
    content2 = fp.readlines()

In [ ]:
len(content2)

9440

In [ ]:
content2[0]

'pattymchugh1212\t41.1179365\t-77.60468399999999\tRT @MartynMcL: Nearly 1,200 days ago, I submitted the first in a series of freedom of information requests about US Secret Service spending ||| RT @mmpadellan: This great patriot @afrosabi, canvassing Georgia with Andrew Yang, has reported that many Republicans are voting for Warnoc ||| RT @JohnFetterman: Nobody knows Biden won more than Mr. Giuliani himself.    My dude, who is 1-46 in court- also refuses to be under oa ||| RT @StephenKing: Yes, the books were excellent. ||| Nope  ||| RT @moot_musings: @ReesusP @StephenKing Why does she always look like shes dressed in stolen and repurposed drapes ||| RT @ReesusP: crap, is this @StephenKings new movie   ||| RT @StephenKing: ELECT STUPID PEOPLE is the Florida state motto. |||   ||| RT @KamalaHarris: 46 days until @JoeBiden becomes our 46th president. ||| RT @LouiseMensch: @RepRiggleman @ninaandtito Thank you sir. Country over party. ||| RT @RepRiggleman: Denver L. Riggleman III  Im on th

In [ ]:
content2[4]

'grey_geena\t32.6782075\t-83.1738665\t@RevDrBarber @Jeffdc5 Take what we can now, then fight like hell in Georgia @fairfightaction ||| RT @RevDrBarber: The Republicans who have forced this bill down to 900 billion are sinful and shameful. ||| @tify330 They are blocked. ||| RT @newsbreakApp: Folks loving Sasha Obamas TikTok dance and long nails video ||| @padakitty @MDogee @briebriejoy @ClimateHuman At which point, Dems will be blamed and GOP back in power in four years. ||| No ones doing anything about it.  ||| RT @MeidasTouch: We are putting this ad up on TV multiple times around the Georgia Senate debate on Sunday. We need to raise 81,590 for th ||| RT @DearDean22:   ||| RT @_DionneWarwick: Hi, @chancetherapper. If you are very obviously a rapper why did you put it in your stage name I cannot stop thinking ||| @PettyHeadlines1 Its like that everyday. ||| @notcapnamerica Saddens me. I dont understand this world. ||| @MDogee @padakitty @briebriejoy @ClimateHuman I dont get it, either. 